In [1]:
import os
import tqdm
import numpy as np
import pandas as pd
from typing import Tuple
from pycatch22 import catch22_all
from joblib import Parallel, delayed
from scipy.signal import argrelextrema
from statsmodels.tsa.stl._stl import STL
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler

In [4]:
DEFAULT_PERIODS = [4, 7, 12, 24, 48, 52, 96, 144, 168, 336, 672, 1008, 1440]

In [3]:
class TimeSeriesFeatureExtractor:

    def __init__(self): pass

    def read_data(self, path: str, nrows=None) -> pd.DataFrame:
        """
        Read the data file and return DataFrame.
        According to the provided file path, read the data file and return the corresponding DataFrame.
        :param path: The path to the data file.
        :return: The DataFrame of the content of the data file.
        """
        data = pd.read_csv(path)
        label_exists = "label" in data["cols"].values

        all_points = data.shape[0]
        columns = data.columns

        if columns[0] == "date":
            n_points = data.iloc[:, 2].value_counts().max()
        else:
            n_points = data.iloc[:, 1].value_counts().max()

        is_univariate = n_points == all_points
        n_cols = all_points // n_points
        df = pd.DataFrame()
        cols_name = data["cols"].unique()

        if columns[0] == "date" and not is_univariate:
            df["date"] = data.iloc[:n_points, 0]
            col_data = {
                cols_name[j]: data.iloc[j * n_points: (j + 1) * n_points, 1].tolist()
                for j in range(n_cols)
            }
            df = pd.concat([df, pd.DataFrame(col_data)], axis=1)
            df["date"] = pd.to_datetime(df["date"])
            df.set_index("date", inplace=True)

        elif columns[0] != "date" and not is_univariate:
            col_data = {
                cols_name[j]: data.iloc[j * n_points: (j + 1) * n_points, 0].tolist()
                for j in range(n_cols)
            }
            df = pd.concat([df, pd.DataFrame(col_data)], axis=1)

        elif columns[0] == "date" and is_univariate:
            df["date"] = data.iloc[:, 0]
            df[cols_name[0]] = data.iloc[:, 1]
            df["date"] = pd.to_datetime(df["date"])
            df.set_index("date", inplace=True)

        else:
            df[cols_name[0]] = data.iloc[:, 0]

        if label_exists:
            last_col_name = df.columns[-1]
            df.rename(columns={last_col_name: "label"}, inplace=True)
            df = df.drop(columns='label')

        if nrows is not None and isinstance(nrows, int) and df.shape[0] >= nrows:
            df = df.iloc[:nrows, :]

        return df

    def feature_extract(self, path: str) -> pd.DataFrame:
        
        # Чтение исходной таблицы с данными
        df = self.read_data(path)
    
        result_rows = []
        feature_vectors = {}

        for col in tqdm.tqdm(df.columns):
            series = df[col].dropna()
            series_length = len(series)

            # Stationarity (ADF)
            try:
                adf_p = adfuller(series.values, autolag="AIC")[1]
            except:
                adf_p = np.nan

            # Extract periods via FFT
            try:
                periods, amplitudes = self.fft_transfer(series.values, fmin=0)
                sorted_amplitudes = sorted(amplitudes, reverse=True)
                periods_list = []

                for amp in sorted_amplitudes:
                    idx = amplitudes.tolist().index(amp)
                    p = int(round(periods[idx]))
                    p = self.adjust_period(p)
                    if p >= 4 and p not in periods_list:
                        periods_list.append(p)
            except:
                periods_list = []

            # Add default periods and deduplicate
            final_periods = periods_list[:3] + DEFAULT_PERIODS
            unique_periods = []
            for p in final_periods:
                if p >= 4 and p not in unique_periods:
                    unique_periods.append(p)

            # STL decomposition for valid periods
            yuzhi = max(series_length // 3, 12)
            seasonality_candidates = {}

            for period in unique_periods:
                if period < yuzhi:
                    try:
                        res = STL(series, period=period).fit()
                        temp_df = pd.DataFrame({
                            "original": series,
                            "trend": res.trend,
                            "seasonal": res.seasonal,
                            "resid": res.resid
                        })
                        temp_df["detrend"] = temp_df["original"] - temp_df["trend"]
                        temp_df["deseasonal"] = temp_df["original"] - temp_df["seasonal"]

                        trend_strength = 0 if temp_df["deseasonal"].var() == 0 else \
                            max(0, 1 - temp_df["resid"].var() / temp_df["deseasonal"].var())
                        seasonal_strength = 0 if temp_df["detrend"].var() == 0 else \
                            max(0, 1 - temp_df["resid"].var() / temp_df["detrend"].var())

                        seasonality_candidates[seasonal_strength] = (seasonal_strength, trend_strength)
                    except:
                        continue

            # Если недостаточно сезонностей — добавим фиктивные
            if len(seasonality_candidates) < 3:
                for i in range(3 - len(seasonality_candidates)):
                    seasonality_candidates[0.1 * (i + 1)] = (-1, -1)

            # Отсортировать по силе сезонности
            sorted_strengths = sorted(seasonality_candidates.items(), key=lambda x: x[0], reverse=True)
            top_strengths = sorted_strengths[:3]

            seasonal_strength1 = top_strengths[0][1][0]
            trend_strength1 = top_strengths[0][1][1]

            # get catch22-based features
            try:
                catch_features = catch22_all(series.values)
                transition = catch_features["values"][catch_features["names"].index("SB_TransitionMatrix_3ac_sumdiagcov")]
                shifting = catch_features["values"][catch_features["names"].index("DN_OutlierInclude_p_001_mdrmd")]
                feature_vectors[col] = np.array(catch_features["values"])
            except:
                transition = np.nan
                shifting = np.nan
                feature_vectors[col] = np.full(22, np.nan)

            result_rows.append({
                "Variable": col,
                "Transition": transition,
                "Shifting": shifting,
                "Seasonality": seasonal_strength1,
                "Trend": trend_strength1,
                "Stationarity": adf_p
            })

        df_result = pd.DataFrame(result_rows)

        # Correlation between time series (via catch22 embeddings)
        try:
            embeddings = np.array([feature_vectors[col] for col in df.columns])

            scaler = MinMaxScaler()
            embeddings_scaled = scaler.fit_transform(embeddings)

            n = embeddings.shape[0]
            corr_list = []
            for i in range(n):
                for j in range(i + 1, n):
                    r = abs(np.corrcoef(embeddings_scaled[i], embeddings_scaled[j])[0, 1])
                    corr_list.append(r)

            if corr_list:
                mean_corr = np.mean(corr_list)
                var_corr = np.var(corr_list)
                #correlation = mean_corr + 1 / (1 + var_corr)
                correlation = 2 * (mean_corr + 1 / (var_corr + 2)) / 3
            else:
                correlation = np.nan
        except:
            correlation = np.nan

        df_result["Correlation"] = correlation

        return df_result

    def fft_transfer(self, timeseries: np.ndarray, fmin: float = 0.2) -> Tuple[np.ndarray, np.ndarray]:
        """
        Perform Fast Fourier Transform on time series.
        :param timeseries: Input time series data
        :param fmin: Minimum frequency threshold
        :return: Tuple of periods and amplitudes
        """
        yf = abs(np.fft.fft(timeseries))
        yfnormlize = yf / len(timeseries)
        yfhalf = yfnormlize[: len(timeseries) // 2] * 2

        fwbest = yfhalf[argrelextrema(yfhalf, np.greater)]
        xwbest = argrelextrema(yfhalf, np.greater)

        fwbest = fwbest[fwbest >= fmin].copy()

        return len(timeseries) / xwbest[0][: len(fwbest)], fwbest

    def adjust_period(self, period_value: int) -> int:
            """
            Adjust period value to nearest standard period.
            Maps the input period to the closest standard period value.
            :param period_value: The input period value to adjust
            :return: The adjusted standard period value
            """
            if abs(period_value - 4) <= 1:
                return 4
            if abs(period_value - 7) <= 1:
                return 7
            if abs(period_value - 12) <= 2:
                return 12
            if abs(period_value - 24) <= 3:
                return 24
            if abs(period_value - 48) <= 1 or ((48 - period_value) <= 4 and (48 - period_value) >= 0):
                return 48
            if abs(period_value - 52) <= 2:
                return 52
            if abs(period_value - 96) <= 10:
                return 96
            if abs(period_value - 144) <= 10:
                return 144
            if abs(period_value - 168) <= 10:
                return 168
            if abs(period_value - 336) <= 50:
                return 336
            if abs(period_value - 672) <= 20:
                return 672
            if abs(period_value - 720) <= 20:
                return 720
            if abs(period_value - 1008) <= 100:
                return 1008
            if abs(period_value - 1440) <= 200:
                return 1440
            if abs(period_value - 8766) <= 500:
                return 8766
            if abs(period_value - 10080) <= 500:
                return 10080
            if abs(period_value - 21600) <= 2000:
                return 21600
            if abs(period_value - 43200) <= 2000:
                return 43200
            return period_value

In [4]:
filepath = './samples/METR-LA.csv'

In [5]:
feature_extractor = TimeSeriesFeatureExtractor()
features = feature_extractor.feature_extract(filepath)

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:32<00:00,  4.61s/it]


In [2]:
class TimeSeriesFeatureExtractorParallel:

    def __init__(self, n_jobs: int = -1):
        self.n_jobs = n_jobs

    def read_data(self, path: str, nrows=None) -> pd.DataFrame:
        data = pd.read_csv(path)
        label_exists = "label" in data["cols"].values
        all_points = data.shape[0]
        columns = data.columns

        if columns[0] == "date":
            n_points = data.iloc[:, 2].value_counts().max()
        else:
            n_points = data.iloc[:, 1].value_counts().max()

        is_univariate = n_points == all_points
        n_cols = all_points // n_points
        df = pd.DataFrame()
        cols_name = data["cols"].unique()

        if columns[0] == "date" and not is_univariate:
            df["date"] = data.iloc[:n_points, 0]
            col_data = {
                cols_name[j]: data.iloc[j * n_points: (j + 1) * n_points, 1].tolist()
                for j in range(n_cols)
            }
            df = pd.concat([df, pd.DataFrame(col_data)], axis=1)
            df["date"] = pd.to_datetime(df["date"])
            df.set_index("date", inplace=True)

        elif columns[0] != "date" and not is_univariate:
            col_data = {
                cols_name[j]: data.iloc[j * n_points: (j + 1) * n_points, 0].tolist()
                for j in range(n_cols)
            }
            df = pd.concat([df, pd.DataFrame(col_data)], axis=1)

        elif columns[0] == "date" and is_univariate:
            df["date"] = data.iloc[:, 0]
            df[cols_name[0]] = data.iloc[:, 1]
            df["date"] = pd.to_datetime(df["date"])
            df.set_index("date", inplace=True)

        else:
            df[cols_name[0]] = data.iloc[:, 0]

        if label_exists:
            last_col_name = df.columns[-1]
            df.rename(columns={last_col_name: "label"}, inplace=True)
            df = df.drop(columns='label')

        if nrows is not None and isinstance(nrows, int) and df.shape[0] >= nrows:
            df = df.iloc[:nrows, :]

        return df

    def fft_transfer(self, timeseries: np.ndarray, fmin: float = 0.2):
        yf = abs(np.fft.fft(timeseries))
        yfnorm = yf / len(timeseries)
        yfhalf = yfnorm[:len(timeseries) // 2] * 2
        fwbest = yfhalf[argrelextrema(yfhalf, np.greater)]
        xwbest = argrelextrema(yfhalf, np.greater)
        fwbest = fwbest[fwbest >= fmin].copy()
        return len(timeseries) / xwbest[0][:len(fwbest)], fwbest

    def adjust_period(self, p):
        for std in DEFAULT_PERIODS:
            if abs(p - std) <= max(1, 0.05 * std):
                return std
        return p

    def process_column(self, col_name, series):
        try:
            adf_p = adfuller(series.values, autolag="AIC")[1]
        except:
            adf_p = np.nan

        try:
            periods, amplitudes = self.fft_transfer(series.values)
            sorted_amps = sorted(amplitudes, reverse=True)
            periods_list = []
            for amp in sorted_amps:
                idx = amplitudes.tolist().index(amp)
                p = int(round(periods[idx]))
                p = self.adjust_period(p)
                if p >= 4 and p not in periods_list:
                    periods_list.append(p)
        except:
            periods_list = []

        final_periods = periods_list[:3] + DEFAULT_PERIODS
        unique_periods = list({p for p in final_periods if p >= 4})

        yuzhi = max(len(series) // 3, 12)
        seasonality_candidates = {}
        for period in unique_periods:
            if period < yuzhi:
                try:
                    res = STL(series, period=period).fit()
                    orig, trend, seasonal, resid = series, res.trend, res.seasonal, res.resid
                    detrend = orig - trend
                    deseasonal = orig - seasonal
                    trend_strength = 0 if deseasonal.var() == 0 else max(0, 1 - resid.var() / deseasonal.var())
                    seasonal_strength = 0 if detrend.var() == 0 else max(0, 1 - resid.var() / detrend.var())
                    seasonality_candidates[seasonal_strength] = (seasonal_strength, trend_strength)
                except:
                    continue

        if len(seasonality_candidates) < 3:
            for i in range(3 - len(seasonality_candidates)):
                seasonality_candidates[0.1 * (i + 1)] = (-1, -1)

        sorted_strengths = sorted(seasonality_candidates.items(), key=lambda x: x[0], reverse=True)
        top_strengths = sorted_strengths[:3]
        seasonal_strength1 = top_strengths[0][1][0]
        trend_strength1 = top_strengths[0][1][1]

        try:
            catch = catch22_all(series.values)
            transition = catch["values"][catch["names"].index("SB_TransitionMatrix_3ac_sumdiagcov")]
            shifting = catch["values"][catch["names"].index("DN_OutlierInclude_p_001_mdrmd")]
            features = np.array(catch["values"])
        except:
            transition = np.nan
            shifting = np.nan
            features = np.full(22, np.nan)

        return {
            "Variable": col_name,
            "Transition": transition,
            "Shifting": shifting,
            "Seasonality": seasonal_strength1,
            "Trend": trend_strength1,
            "Stationarity": adf_p
        }, features

    def feature_extract(self, path: str) -> pd.DataFrame:
        df = self.read_data(path)
        results = Parallel(n_jobs=self.n_jobs)(
            delayed(self.process_column)(col, df[col].dropna()) for col in df.columns
        )
        rows, feature_vectors = zip(*results)
        df_result = pd.DataFrame(rows)

        try:
            embeddings = np.stack(feature_vectors)
            scaler = MinMaxScaler()
            embeddings_scaled = scaler.fit_transform(embeddings)

            n = embeddings_scaled.shape[0]
            corr_list = [abs(np.corrcoef(embeddings_scaled[i], embeddings_scaled[j])[0, 1])
                         for i in range(n) for j in range(i + 1, n)]
            if corr_list:
                mean_corr = np.mean(corr_list)
                var_corr = np.var(corr_list)
                correlation = 2 * (mean_corr + 1 / (var_corr + 2)) / 3
            else:
                correlation = np.nan
        except:
            correlation = np.nan

        df_result["Correlation"] = correlation
        return df_result

In [5]:
extractor = TimeSeriesFeatureExtractorParallel(n_jobs=-1)

filepath = './datasets/ETTh1.csv'
result_df = extractor.feature_extract(filepath)

In [6]:
result_df.to_csv('./characteristics/TFB_characteristics_ETTh1.csv', index=None)

In [7]:
result_df

,Variable,Transition,Shifting,Seasonality,Trend,Stationarity,Correlation
0,HUFL,0.002915,0.089093,0.888109,0.735468,9.250276e-14,0.630153
1,HULL,0.034014,0.147417,0.758769,0.912932,1.016534e-05,0.630153
2,MUFL,0.001902,0.071929,0.888785,0.718622,6.098079e-14,0.630153
3,MULL,0.020576,-0.026005,0.748805,0.916134,2.612875e-05,0.630153
4,LUFL,0.020314,0.076808,0.686989,0.777871,4.728964e-07,0.630153
5,LULL,0.012850,0.117222,0.543156,0.925236,6.132123e-05,0.630153
6,OT,0.046296,-0.905999,0.600216,0.988229,8.301649e-03,0.630153


In [20]:
result_df.to_csv('./features.csv', index=None)

In [21]:
result_df

,Variable,Transition,Shifting,Seasonality,Trend,Stationarity,Correlation
0,773869,0.000977,-0.025444,0.463696,0.106712,3.346036e-22,0.778835
1,767541,0.004723,0.038690,0.493555,0.052963,2.673162e-24,0.778835
2,767542,0.002399,0.026669,0.546320,0.559038,8.830922e-26,0.778835
3,717447,0.000749,0.043009,0.515884,0.053369,5.204197e-28,0.778835
4,717446,0.002385,0.011569,0.637306,0.478229,8.544499e-30,0.778835
...,...,...,...,...,...,...,...
202,717592,0.004315,-0.035510,0.413168,0.081001,7.214606e-20,0.778835
203,717595,0.007999,0.029645,0.490262,0.049153,8.247571e-25,0.778835
204,772168,0.001061,-0.025137,0.518219,0.044431,2.780529e-30,0.778835
205,718141,0.002366,0.025268,0.471262,0.033157,2.694505e-28,0.778835


In [34]:
features#.select_dtypes('float').mean(axis=0)

,Variable,Transition,Shifting,Seasonality,Trend,Stationarity,Correlation
0,HUFL,0.020576,-0.132346,0.778980,0.874422,0.002327,0.52759
1,HULL,0.036458,0.066173,0.711274,0.523518,0.028170,0.52759
2,MUFL,0.004372,-0.053679,0.793127,0.849772,0.000098,0.52759
3,MULL,0.040000,0.146691,0.699340,0.627387,0.075278,0.52759
4,LUFL,0.111111,-0.130032,0.713562,0.791128,0.000589,0.52759
5,LULL,0.010000,-0.824155,0.662350,0.534879,0.000865,0.52759
6,OT,0.111111,-0.405831,0.701958,0.968497,0.311402,0.52759


In [39]:
# Требуемый результат
pd.read_csv('./characteristics/TFB_characteristics_ETTh1.csv')

,Correlation,Transition,Shifting,Seasonality,Trend,Stationarity,Short_term_jsd,Long_term_jsd
0,NaN,0.020576,0.132346,0.778980,0.874422,0.002327,0.069880,0.043828
1,NaN,0.036458,0.066173,0.711274,0.523518,0.028170,0.062927,0.074604
2,NaN,0.004372,0.053679,0.793127,0.849772,0.000098,0.089969,0.052785
3,NaN,0.040000,0.146691,0.699340,0.627387,0.075278,0.072865,0.061634
4,NaN,0.111111,0.130032,0.713562,0.791128,0.000589,0.071073,0.074040
5,NaN,0.010000,0.824155,0.662350,0.534879,0.000865,0.316538,0.259594
6,NaN,0.111111,0.405831,0.701958,0.968497,0.311402,0.052107,0.046819


In [13]:
# Требуемый результат
pd.read_csv('./characteristics/mean_TFB_characteristics_ETTh1.csv')

,Correlation,Transition,Shifting,Seasonality,Trend,Stationarity,Short_term_jsd,Long_term_jsd
0,0.52759,0.047661,0.251273,0.722942,0.738515,0.059819,0.105051,0.087615


In [12]:
all_sets = ['Electricity.csv', 'METR-LA.csv', 'PEMS-BAY.csv', 'PEMS04.csv', 'PEMS08.csv', 'Solar.csv']

# Теперь считаем срдение характеристики для БД
for filename in all_sets:
    characteristics = pd.read_csv(f'./characteristics/TFB_characteristics_{filename}')
    characteristics = characteristics.set_index('Variable')

    print(filename.split('.')[0], characteristics.abs().mean().round(3).to_dict())

Electricity {'Transition': 0.01, 'Shifting': 0.194, 'Seasonality': 0.94, 'Trend': 0.81, 'Stationarity': 0.005, 'Correlation': 0.802}
METR-LA {'Transition': 0.004, 'Shifting': 0.03, 'Seasonality': 0.521, 'Trend': 0.269, 'Stationarity': 0.0, 'Correlation': 0.779}
PEMS-BAY {'Transition': 0.005, 'Shifting': 0.09, 'Seasonality': 0.629, 'Trend': 0.145, 'Stationarity': 0.0, 'Correlation': 0.842}
PEMS04 {'Transition': 0.007, 'Shifting': 0.085, 'Seasonality': 0.917, 'Trend': 0.339, 'Stationarity': 0.0, 'Correlation': 0.797}
PEMS08 {'Transition': 0.005, 'Shifting': 0.095, 'Seasonality': 0.913, 'Trend': 0.376, 'Stationarity': 0.0, 'Correlation': 0.807}
Solar {'Transition': 0.022, 'Shifting': 0.058, 'Seasonality': 0.919, 'Trend': 0.478, 'Stationarity': 0.0, 'Correlation': 0.753}
